In [17]:
# sudo pip install --upgrade --force-reinstall numpy
# sudo pip install --upgrade --force-reinstall scipy
# sudo pip install --upgrade --force-reinstall h5py
# sudo pip install --upgrade --force-reinstall keras
# sudo pip install --upgrade --force-reinstall tensorflow
# conda install keras
# conda install opencv

from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [19]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [20]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    '''
    implementation of the triplet loss as defined by the formula
    y_true = true labels, required when you define a loss in Keras, you don't need it in this function
    y_pred = python list that has three objects
        anchor = the encodings for the anchor images, of shape (None, 128)
        positive = the encodings for the positive images, of shape (None, 128)
        negative = the encodings for the negative images, of shape (None, 128)
    Returns:
    loss = real number, value of the loss
    '''
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # compute the distance between the anchor and the positive, you will need to sum over axis = -1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
    
    # compute the distance between the anchor and the negative, you will need to sum over axis = -1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
    
    # subtract the two previous distances and add alpha
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    
    # take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

In [21]:
# unit test for triplet_loss method
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed=1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed=1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed=1))
    loss = triplet_loss(y_true, y_pred)
    print("expect loss " + str(loss.eval()) + " to be " + str(528.1427))

expect loss 528.1427 to be 528.1427


In [22]:
FRmodel.compile(optimizer = "adam", loss = triplet_loss, metrics = ["accuracy"])
load_weights_from_FaceNet(FRmodel)

In [23]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

In [25]:
def verify(image_path, identity, database, model):
    """
    function that verifies if the person on the image path image is 
    identity
    
    Arguments: 
    image_path = path to image
    identity = string, name of the person you'd like to verify the identity 
    database = python dictionary mapping names of allowed people's names
    model = your Inception mode instance in keras 
    
    Returns:
    dist = distance between the image_path and the image of identity in db
    door_open = true if the door should open, false otherwise
    """
    
    # step 1: compute the encoding for the image, use img_to_encoding
    encoding = img_to_encoding(image_path, model)
    
    # step 2: compute distance with identity's image
    dist = np.linalg.norm(encoding - database[identity], ord=2)
    
    # step 3: open the door if dist < 0.7, else don't open 
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else: 
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    return dist, door_open

In [26]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

It's younes, welcome home!


(0.65938437, True)

In [27]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

It's not kian, please go away


(0.8622572, False)

In [28]:
def who_is_it(image_path, database, model):
    """
    implements face recognition for the house by finding who is the person
    on the image path
    
    Arguments:
    image_path = path to an image 
    database = database containing image encodings along with the name of 
    the person on the image
    model = your Inception model instance in Keras 
    
    Returns:
    min_dist = the minimum distance between image_path encoding and the 
    encodings from the database
    identity = string, the name prediction for the person on image_path
    """
    
    # step 1: compute the target encoding for the image
    encoding = img_to_encoding(image_path, model)
    
    # step 2: finding the closest encoding 
    # initialize min_dist to a large value, say 100
    min_dist = 100
    
    # loop over the database dictionary's names and encodings 
    for (name, db_enc) in database.items():
        
        # compute L2 distance between the target encoding and the current
        # emb from the database 
        dist = np.linalg.norm(db_enc - encoding, ord=2)
        
        # if this distance is less than the min_dist, then set min_dist to
        # dist, and identity to name
        if dist < min_dist:
            min_dist = dist
            identity = name 
            
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print("it's " + str(identity) + ", the distance is " + str(min_dist))
    
    return min_dist, identity

In [29]:
who_is_it("images/camera_0.jpg", database, FRmodel)

it's younes, the distance is 0.65938437


(0.65938437, 'younes')